# Sales Bot with Llama3 - A Summarization and RAG Use Case

## Overview

In this notebook you'll take an Amazon product reviews dataset from Kaggle and use Llama3 to obtain product review summaries, upsert those summaries in a vector database, then use Retrieval Augmented Generation (RAG) to power a sales chatbot that can make targeted product recommendations.

Let's take a look at the overall workflow:
1. We start with a dataset that contains over 10,000 reviews across 900 Amazon musical instruments and accessories.
2. Using Llama2 70B chat (hosted on OctoAI), we generate summaries of product reviews for each product from the 20 most recent reviews. We format the summaries in JSON format.
3. We then take the summaries and upsert them into a vector database (Weaviate in this case)
4. We then use this vector database and Llama3 70B instruct (hosted on OctoAI) to build a RAG-based sales chatbot that provides targeted recommendations to the user based on the products that are present in the inventory.

Note: at the time of writing this tutorial, JSON mode formatting isn't supported for Llama 3 on OctoAI via constrained sampling which is why we are falling back onto Llama 2. This tutorial will be updated when the feature becomes available to rely on Llama 3 exclusively.

### OctoAI
We'll use [OctoAI](https://octo.ai/) to power all of the GenAI model needs of this notebook: LLMs, image gen, image animation.
* To use OctoAI, you'll need to go to https://octoai.cloud/ and sign in using your Google or GitHub account.
* Next you'll need to generate an OctoAI API token by following these [instructions](https://octo.ai/docs/getting-started/how-to-create-an-octoai-access-token). Keep the API token in hand, we'll need it further down in this notebook.

In this example we will use the Llama 3 70b instruct model. You can find more on Llama models on the [OctoAI text generation solution page](https://octoai.cloud/text).

At the time of writing this notebook the following Llama models are available on OctoAI:
* meta-llama-3-8b-instruct
* meta-llama-3-70b-instruct
* codellama-7b-instruct
* codellama-13b-instruct
* codellama-34b-instruct
* llama-2-13b-chat
* llama-2-70b-chat
* llamaguard-7b

### Weaviate
We'll use Weaviate Cloud Services (WCS) for our vector database. You can create an account and Weaviate clusters easily at the following link: https://console.weaviate.cloud/.
You can then create a cluster, from which you can obtain the REST Endpoint URL and the API key to use the cluster endpoint.

### OpenAI
We'll be using OpenAI for its embedding model to upsert our vectors into the Weaviate vector database. Create an account and obtain an API key here: https://openai.com/api/

### Local Python Notebook
We highly recommend launching this notebook from a fresh python environment, for instance you can run the following:
```
python3 -m venv .venv         
source .venv/bin/activate
```
All you need to run this notebook is to install jupyter notebook with `python3 -m pip install notebook` then run `jupyter notebook` ([link](https://jupyter.org/install)) in the same directory as this `.ipynb` file.
You don't need to install additional pip packages ahead of running the notebook, since those will be installed right at the beginning. You will need to ensure your system has `imagemagick` installed by following the [instructions](https://imagemagick.org/script/download.php).

In [ ]:
# Let's start by installing the appropriate python packages
! pip install octoai===1.0.2 openai weaviate-client pandas gradio pydantic

## Part 1: Review Summarization

Let's start by importing all of the packages we need for this example

In [ ]:
import gradio
import json
import langchain
import os
import openai
import weaviate
from getpass import getpass
from json import loads
from pandas import DataFrame, concat, read_csv
from pydantic import BaseModel, Field
from typing import List
import weaviate.classes as wvc

Enter your OctoAI, Weaviate, and OpenAI tokens below

In [ ]:
# Get OctoAI API token for Llama 2 & 3
OCTOAI_API_TOKEN = getpass()
os.environ["OCTOAI_API_TOKEN"] = OCTOAI_API_TOKEN

In [ ]:
# Get WCS API key
WCS_API_KEY = getpass()
os.environ["WCS_API_KEY"] = WCS_API_KEY

In [ ]:
# Get WCS URL
WCS_URL = getpass()
os.environ["WCS_URL"] = WCS_URL

In [ ]:
# Get OpenAI API key for the embedding model
OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
# First let's load the dataset from Kaggle: https://www.kaggle.com/datasets/eswarchandt/amazon-music-reviews
df = read_csv('Musical_instruments_reviews.csv')

Set `product_record_limit` to a lower number if you just want to do a test run

In [ ]:
# Set a product record limit
product_record_limit = 900

# List all of the unique ASIN:
asin_list = df.asin.unique()
print("There are {} unique products in the music product inventory".format(len(asin_list)))

For each one of the unique products, let's group the reviews together and sort them by how recent they are

In [ ]:
# Get the reviews for the product ASIN, sorted by recency and store in dict
review_dict = {}
for asin in asin_list[0:product_record_limit]:
    reviews = df.loc[df['asin'] == asin]\
                .sort_values(["unixReviewTime"], axis=0, ascending=False)\
                .reviewText.tolist()
    review_dict[asin] = reviews

To be able to store our summaries into our vector DB, we need to have the fields formatted into a JSON object. We use Pydantic base class model here to define our formatting.

In [ ]:
# Define the Pydantic model that specifies how our output should be formatted
class ProductRecord(BaseModel):
    """The record of a given product"""
    description: str = Field(description="Description of the product")
    name: str = Field(description="Name of the product")
    review_summary: str = Field(description="Summary of all of the reviews")
    ASIN: str = Field(description="ASIN of the product")
    features: str = Field(description="Features of the product based on the reviews")

We define our prompt template below.

In [ ]:
# Prepare a prompt template
template = '''
Here are product reviews for a music product with an ID of {asin}.
 - Respond back only as only JSON!
 - Provide:
     - the product "description",
     - the product "name",
     - a summary of all the reviews as "review_summary",
     - the "ASIN" and
     - and the product "features" based on the content of these reviews. 
 - The "features" should be a string describing the features and NOT JSON. 
 - Do not include the ASIN in the description field.
 
The reviews for the product are: {reviews}
'''

We initialize the OctoAI client using OpenAI's API. All we have to do is override the `base_url` and `api_key`.

In [ ]:
# Init OctoAI client
client = openai.OpenAI(
    base_url="https://text.octoai.run/v1",
    api_key=os.environ["OCTOAI_API_TOKEN"]
)

Iterate over all product ASINs and summarize the top 20 most recent reviews. Note: this takes a while to run unless we parallelize it.

In [ ]:
# Produce the 900 product summaries
review_summaries = []
counter = 0

# This can take a while to process serially (30min+)
# TODO: Optimize to run in a few parallel threads to run faster while meeting the 240RPM limit
for asin, review_list in review_dict.items():
    print(f'Getting review summary {counter} of {len(review_dict)}, ASIN: {asin}')
    try:
        response = client.chat.completions.create(
            model="llama-2-70b-chat",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": template.format(
                    asin = asin,
                    reviews = review_list[0:20]
                )},
            ],
            temperature=0,
            response_format={"type": "json_object", "schema": ProductRecord.model_json_schema()},
            max_tokens=1024
        )
        print("\n{}\n".format(response.choices[0].message.content))
        summary = loads(response.choices[0].message.content)
        summary["ASIN"] = asin
        review_summaries.append(summary)
    except:
        print(f'Issue with ASIN {asin}, skipping')
        pass
    counter += 1

review_summaries = DataFrame(review_summaries)

print(review_summaries)

# Part 2: Retrieval Augmented Generation

For our RAG use case we're going to rely on Weaviate vector database and on an OpenAI embedding model. 

When you define your collection, you'll need to provide properties, i.e. object attributes that you want to store in the collection. These properties map 1:1 to the JSON dictionary keys defined earlier for the `ProductRecord` Pydantic base model.

In [ ]:
# Connect to WCS
wcs_client = weaviate.connect_to_wcs(
    cluster_url=os.getenv("WCS_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WCS_API_KEY")),
    headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"]
    }
)

In [ ]:
# Create the collection if it doesn't already exist
try:
    collection = wcs_client.collections.get("Products")
except:
    # Create the collection for products
    collection = wcs_client.collections.create(
        name="Products",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
        properties=[
            wvc.config.Property(
                name="ASIN",
                data_type=wvc.config.DataType.TEXT
            ),
             wvc.config.Property(
                name="name",
                data_type=wvc.config.DataType.TEXT
            ),
             wvc.config.Property(
                name="review_summary",
                data_type=wvc.config.DataType.TEXT
            ),
             wvc.config.Property(
                name="features",
                data_type=wvc.config.DataType.TEXT
            ),
             wvc.config.Property(
                name="description",
                data_type=wvc.config.DataType.TEXT
            ),
        ]
    )
    print("Collection Created!")

Now we upsert all of the vectors into the databse using OpenAI's embedding model.

In [ ]:
# Convert df to JSON string and then to a list of dictionaries
data = review_summaries.to_json(orient='records')
data_list = json.loads(data)

items_to_insert = []

for d in data_list:
    new_item = {
        "ASIN": d["ASIN"],
        "name": d["name"],
        "description": d["description"],  \
        "features": d["features"],
        "review_summary": d["review_summary"]
    }
    items_to_insert.append(new_item)

    # Insert every 100 items
    if len(items_to_insert) == 100:
        collection.data.insert_many(items_to_insert)
        items_to_insert.clear()

# Insert remaining items
if len(items_to_insert) > 0:
    collection.data.insert_many(items_to_insert)

Let's now try to run a hybrid search on the following query below.
Hybrid search combines the results of a vector search and a keyword (BM25F) search by fusing the two result sets.
It will return the 3 closest entries in the database according to the search criteria.

In [ ]:
# Hybrid search
response = collection.query.hybrid(
    query="easy to learn instrument",
    limit=3
)
for o in response.objects:
    print(o.properties)

Let's now define a helper function that gives us the relevant context given a string query. Let's see what it returns based on the question: "What is a good beginner harmonica"

In [ ]:
# Helper function to run hybrid search on a user query and return the closest
# product review summaries relevant to the user query
def get_context(question, limit=3):
    response = collection.query.hybrid(
        query=question,
        limit=limit
    )
    return "\n".join([str(o.properties) for o in response.objects])

print(get_context("What is a good beginner harmonica"))

Great, we're now ready to build a sales assistant helper function.

We first define a prompt template for Llama 3 - based on the context provided by the vector hybrid search (i.e. collection of product summaries of relevance to the question), provide a helpful recommendation to the customer. 

Also provide links to the product that the user can click on to view the product on Amazon's website. For that we use the fact that any product referenced by its aSIN can be accessed at the following url: `https://www.amazon.com/exec/obidos/ASIN/<insert aSIN here>`

In [ ]:
sales_template = """
You are a sales assistant. Answer the user questions as helpfully as possible.
Only recommend the products that are provided in the context provided below.

Provide a reference to each product you mention with hyperlinks:
* Provide the name of the product
* Embed the hyperlink in the name of the product as follows
    * If the product name is "Solid Electric Guitar Case with Accessories Compartment"
    * And the aSIN is "B001EL6I8W"
    * Format the reference as follows: 
         [Solid Electric Guitar Case with Accessories Compartment](https://www.amazon.com/exec/obidos/ASIN/B001EL6I8W)

Finish with a references section.

Customer question: {}

Product context: {}

AI:
"""

def sales_assistant(question):  
    response = client.chat.completions.create(
                model="meta-llama-3-70b-instruct",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": sales_template.format(question, get_context(question, limit=10))},
                ],
                temperature=0,
                max_tokens=1024
            )
    
    return response.choices[0].message.content

print(sales_assistant("what is must have accessory for my new electric guitar"))

# Part 3: Gradio-based sales assistant demo

In this section we build a simple an interactive sales bot assistant using Gradio.

In [ ]:
import gradio as gr

def predict(message, history):
    history_openai_format = []
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human})
        history_openai_format.append({"role": "assistant", "content": assistant})
    history_openai_format.append({"role": "user", "content": sales_template.format(message, get_context(message, limit=5))})

    response = client.chat.completions.create(
        model = 'meta-llama-3-70b-instruct',
        messages = history_openai_format,
        temperature = 0.0,
        stream = True
     )

    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
              partial_message = partial_message + chunk.choices[0].delta.content
              yield partial_message

gr.ChatInterface(predict).launch()

**Authors**
- Thierry Moreau, OctoAI - tmoreau@octo.ai
- Jonathan Tuite, Weaviate - jon@weaviate.io